In [ ]:
from ultralytics import YOLO
from os import path, listdir, makedirs
from shutil import rmtree, copy
import time
import random

In [ ]:
# Constants
CHECKPOINT_PATH = f'./models/checkpoints/'
IMAGE_SIZE = 640 # x 480
PROJECT_NAME = "logs"
RECIPE_NAME = "all"
TRAIN_RUN = f"{RECIPE_NAME}_run-{time.strftime('%Y%m%d-%H%M%S')}"

# Model
MODEL_PATH = f"./models/pretrained/yolov8n-obb-dotav1.pt"
TRAIN_PARAM = {
    # Model definition
    'data': f"./models/recipes/dataset_desc.yaml",
    'resume': False,
    'device': '0',
    'pretrained': True,
    # Names
    'project' : PROJECT_NAME,
    'name': TRAIN_RUN,
    # Training Parameters
    'batch': -1,
    'imgsz': IMAGE_SIZE,
    'epochs': 1,
    'patience': 10,
    'cos_lr': True,
    # Augmentation
    'hsv_h': 0.05, # Higher than default for resistor
    'hsv_s': 0.3, # Colours should not change too much
    'hsv_v': 0.2, # Colours should not change too much
    'degrees': 180, # Rotation
    'translate': 0.1, # Translation
    'scale': 0.8, # Scaling - camera is always at the same distance
    'shear': 10.0, # Shearing
    'perspective': 0.0, # Perspective
    'flipud': 0.5, # Flip up-down
    'fliplr': 0.5, # Flip left-right
    'mosaic': 0.5, # Mosaic
    'mixup': 0.0, # Mixup
    'copy_paste': 0.0, # Copy-paste
    'crop_fraction': 1.0, # Crop fraction
    # Loss weights
    'cls' : 1.0, # Class
    'box' : 4.0, # Box accuracy
    'dfl' : 1.5, # Help manage unbalanced classes
    # Post parameters
    'save': True,
    'save_period': 5,
    'plots': False,
    # Misc
    'verbose': False,
}


In [ ]:
# Reorganise the data
PARAM = {
    'path' : "./datasets",
    'train' : 0.7,
    'val' : 0.2,
    'test' : 0.1,
    'include' : [
        'resistor',
        'capacitor',
        'ceramic_capacitor',
        'film_capacitor',
        'inductor',
        'led',
        'wire'
    ]
}
# Remove old dataset
DATASET_FOLDER = f"{PARAM['path']}/current"
if path.exists(DATASET_FOLDER):
    for folder in listdir(DATASET_FOLDER):
        rmtree(path.join(DATASET_FOLDER, folder), ignore_errors=True)
    # Make label folder
    makedirs(path.join(DATASET_FOLDER, 'labels'), exist_ok=True)
    # Make train, val, test folders
    for folder in ['train', 'val', 'test']:
        makedirs(path.join(DATASET_FOLDER, 'images', folder), exist_ok=True)
        makedirs(path.join(DATASET_FOLDER, 'labels', folder), exist_ok=True)

# Get all component images from all folders
basenames = []
# For component folder in dataset
for folder in listdir(PARAM['path']):
    # Skip if not in include
    if folder not in PARAM['include']: continue
    # For each subfolder in component folder
    imgfiles = listdir(path.join(PARAM['path'], folder, 'imgs'))
    labfiles = listdir(path.join(PARAM['path'], folder, 'labels'))
    bases = [path.join(folder, 'imgs', path.splitext(f)[0]) for f in imgfiles]
    basenames.extend(bases)
    print(f"Found {len(imgfiles)} images and {len(labfiles)} labels in {folder}")

# Split the data into train, val, test
random.shuffle(basenames)
train = int(len(basenames) * PARAM['train'])
val = int(len(basenames) * PARAM['val'])
test = int(len(basenames) * PARAM['test'])
print(f"Split into {train} train, {val} val, {test} test. Total: {train+val+test} images.")
train_set = basenames[:train]
val_set = basenames[train:train+val]
test_set = basenames[train+val:]

# Copy the images and labels to the new dataset folder
for folder, dataset in zip(['train', 'val', 'test'], [train_set, val_set, test_set]):
    for base in dataset:
        filename = path.split(base)[1]
        copy(path.join(PARAM['path'], f"{base}.png"), path.join(DATASET_FOLDER, 'images', folder, f"{filename}.png"))
        base = base.replace('imgs', 'labels')
        copy(path.join(PARAM['path'], f"{base}.txt"), path.join(DATASET_FOLDER, 'labels', folder, f"{filename}.txt"))

# # Copy the images and labels to the new dataset folder
# for folder, dataset in zip(['train', 'val', 'test'], [train_set, val_set, test_set]):
#     for base in dataset:
#         filename = path.split(base)[1]
#         copy(path.join(PARAM['path'], f"{base}.png"), path.join(DATASET_FOLDER, folder, 'imgs', f"{filename}.png"))
#         base = base.replace('imgs', 'labels')
#         copy(path.join(PARAM['path'], f"{base}.txt"), path.join(DATASET_FOLDER, folder, 'labels', f"{filename}.txt"))


In [ ]:
# Tensorboard logging
%load_ext tensorboard
%tensorboard --logdir "logs"

In [ ]:
# Train
model = YOLO(MODEL_PATH).to('cuda')
model.train(**TRAIN_PARAM)